In [1]:
# Activate Spark in our Colab notebook.
import os
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-15 18:23:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.05MB/s    in 0.2s    

2023-01-15 18:23:02 (5.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...|2015-01-31 00:08:00|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun.

In [5]:
df.count()

5115666

In [6]:
df = df.dropna()
df.count()

5114733

# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612|2015-01-31 00:08:00|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691|2015-01-31 00:08:00|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471|2015-01-31 00:08:00|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889|2015-01-31 00:08:00|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747|2015-01-31 00:08:00|
|R1R30FA4RB5P54|    2909389|B000NYL1Z6|     166146615|2015-01-31 00:08:00|
|R30IJKCGJBGPJH|   19397215|B001SYWTFG|     111742328|2015-01-31 00:08:00|
|R18GLJJPVQ1OVH|    3195210|B005F2EVMQ|     255803087|2015-01-31 00:08:00|
| R8TVYIJXLYJT0|   52216383|B00M1SUW7K|     246816549|2015-01-31 00:08:00|
|R1CJGF6M3PVHEZ|   10278216|B001KYQA1S|       9612905|2015-01-31 00:08:00|
|R2A30ALEGLMCGN|   246554

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FBO8FCA|Mederma Advanced ...|
|B003EMJHG4|Secret Clinical S...|
|B00BDGXT9Y|Brenda Christian ...|
|B00NW5IIOK|Jewelry Woman New...|
|B004HAMIJA|Mary Kay Oil Free...|
|B005IJQAIE|Piona StrongBrigh...|
|B00JKK6118|Vitabath Moisturi...|
|B00V954IYY|Boar Bristle Hair...|
|B002BAOMAG|Lavanila The Heal...|
|B00KGLNWBW|Natural Life Boho...|
|B00Y1C8P64|L'Oréal Paris Rev...|
|B000KL1N3C|Pill Crusher -Pil...|
|B000PHDEL0|Sally Hansen Comp...|
|B004EFG4BG|Moroccanoil Treat...|
|B00A3BWYU0|36 Color Hair Cha...|
|B00D1FBRIK|4.1" Resin Banana...|
|B001E96M94|Johnson's Body Ca...|
|B00GJ1PYZ0|Escape Aroma Ther...|
|B00LZZ7FZW|Gold Bond Skin Re...|
|B000MASTN8|MyChelle Dermaceu...|
+----------+--------------------+
only showing top 20 rows



In [10]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
# Load in a sql function to use columns
from pyspark.sql.functions import desc
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36771761|           871|
|   32405532|           721|
|   10942711|           685|
|   37446839|           587|
|   50199793|           564|
|   12201275|           519|
|    4808156|           508|
|   48233483|           440|
|   52520442|           378|
|   39789300|           346|
|   18609243|           343|
|   37337835|           337|
|   42799904|           314|
|   21012418|           288|
|   10592389|           284|
|   52433525|           250|
|   51126995|           245|
|   52824002|           235|
|   18715781|           230|
|   31120312|           219|
+-----------+--------------+
only showing top 20 rows



In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3I2DHQBR577SS|          5|            0|          0|   N|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|
| RAI2OIG50KZ43|          5|            0|          0|   N|
|R1R30FA4RB5P54|          4|            0|          0|   N|
|R30IJKCGJBGPJH|          5|            0|          0|   N|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|
| R8TVYIJXLYJT0|          5|            0|          0|   N|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|
|R2A30ALEGLMCGN|          4|            1|          1|   N|
|R134GP1Y3EQZ1L|          5|            0|          0|   N|
| R6CE3SOIUJGP4|          4|            1|          1|   N|
|R37CDLQZUPDZVM|          5|            

## Load

In [17]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "<Password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)